In [42]:
using ITensors, ITensorMPS, Optim, LinearAlgebra, Printf, Random, Plots, DelimitedFiles
include("src/basic_CPHL_utils.jl")


N_g = 30;
g_vals = collect(range(-1, 1; length = N_g + 2)[2:end-1]);

In [ ]:
M_max = 5 
DesignMat, HarmFromVecMat = ConstructDesignMatrix(g_vals, M_max)

CMat = zeros(Float64, length(g_vals), 2)

CMat[:, 1] = sin.(pi * 1 .* (g_vals .+ 1) / 2) + 0.1 * sin.(pi * 2 .* (g_vals .+ 1) / 2) - 0.3 * sin.(pi * 3 .* (g_vals .+ 1) / 2) - 0.13 * sin.(pi * 5 .* (g_vals .+ 1) / 2)
CMat[:, 2] = -0.2 * sin.(pi * 1 .* (g_vals .+ 1) / 2)


ALPHAMat =  zeros(M_max, 2);
ALPHAMat[:, 1] = [1; 0.1; -0.3; 0.0; -0.13]
ALPHAMat[:, 2] = [-0.2; 0; 0; 0; 0]

@show norm(ALPHAMat - EstimateHarmFromCMat(CMat, HarmFromVecMat))

norm(GenerateCMatFromALPHAMat(ALPHAMat, DesignMat) - CMat)

norm(ALPHAMat - EstimateHarmFromCMat(CMat, HarmFromVecMat)) = 6.966282800046645e-14


4.0141141393567177e-16

In [44]:
typeof(g_vals)

Vector{Float64} (alias for Array{Float64, 1})

In [45]:
include("src/CPHL_module.jl")

cphl = CPHL.CPHLSolver(10, g_vals)

Main.CPHL.CPHLSolver(10, Index{Int64}[(dim=2|id=499|"S=1/2,Site,n=1"), (dim=2|id=312|"S=1/2,Site,n=2"), (dim=2|id=679|"S=1/2,Site,n=3"), (dim=2|id=294|"S=1/2,Site,n=4"), (dim=2|id=723|"S=1/2,Site,n=5"), (dim=2|id=634|"S=1/2,Site,n=6"), (dim=2|id=230|"S=1/2,Site,n=7"), (dim=2|id=459|"S=1/2,Site,n=8"), (dim=2|id=771|"S=1/2,Site,n=9"), (dim=2|id=324|"S=1/2,Site,n=10")], [-0.9354838709677419, -0.8709677419354839, -0.8064516129032258, -0.7419354838709677, -0.6774193548387096, -0.6129032258064516, -0.5483870967741935, -0.4838709677419355, -0.41935483870967744, -0.3548387096774194  …  0.3548387096774194, 0.41935483870967744, 0.4838709677419355, 0.5483870967741935, 0.6129032258064516, 0.6774193548387096, 0.7419354838709677, 0.8064516129032258, 0.8709677419354839, 0.9354838709677419], 30, 3, [0.10116832198743225 0.20129852008866023 0.29936312297335815; 0.20129852008866006 0.39435585511331855 0.5712682150947923; … ; 0.20129852008866017 -0.3943558551133187 0.5712682150947918; 0.10116832198743228 